In [1]:
pip install transformers datasets scikit-learn pandas torch

In [2]:
pip install --upgrade transformers


In [3]:
!pip install --upgrade ipywidgets


# Loading data from kaggle

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment-analysis-dataset


In [5]:
import pandas as pd
import os

# Load the CSV from the downloaded path
csv_path = os.path.join(path, "train.csv")
df = pd.read_csv(csv_path, encoding='ISO-8859-1')
print(df.head())


       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment Time of Tweet Age of User  \
0  I`d have responded, if I were going   neutral       morning        0-20   
1                             Sooo SAD  negative          noon       21-30   
2                          bullying me  negative         night       31-45   
3                       leave me alone  negative       morning       46-60   
4                        Sons of ****,  negative          noon       60-70   

       Country  Population -2020  Land Area (Km²)  Density (P/Km²)  
0  Afghanistan          38928346         652860.0    

# Clean & Prepare Data

In [6]:
# Keep only the relevant columns
df = df[['text', 'sentiment']].dropna()

# Map sentiment labels to numerical values
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
df['sentiment'] = df['sentiment'].str.lower().map(label_map)

# Drop rows with unexpected sentiment values
df = df[df['sentiment'].isin([0, 1, 2])]

print(df['sentiment'].value_counts())


sentiment
1    11117
2     8582
0     7781
Name: count, dtype: int64


# Train/Test Split

In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['sentiment'].tolist(),
    test_size=0.2,
    stratify=df['sentiment'],
    random_state=42
)


# Tokenize with BERT

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Prepare Dataset for Training

In [9]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            **{k: torch.tensor(v[idx]) for k, v in self.encodings.items()},
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)


# Fine-tune BERT

In [10]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=200,
    do_eval=True,             # explicitly enable evaluation
    save_total_limit=1        # save only the last checkpoint
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pranavam-caparna (aparnacmi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
200,0.728000
400,0.609400
600,0.566000
800,0.585600
1000,0.556400
1200,0.543500
1400,0.505000
1600,0.387400
1800,0.406700
2000,0.370100


TrainOutput(global_step=4122, training_loss=0.39528944606864297, metrics={'train_runtime': 3196.7565, 'train_samples_per_second': 20.631, 'train_steps_per_second': 1.289, 'total_flos': 3728152683319680.0, 'train_loss': 0.39528944606864297, 'epoch': 3.0})

# Evaluate Model

In [11]:
from sklearn.metrics import classification_report

# Get predictions
preds = trainer.predict(val_dataset)
y_pred = preds.predictions.argmax(-1)

print(classification_report(val_labels, y_pred, target_names=['Negative', 'Neutral', 'Positive']))


              precision    recall  f1-score   support

    Negative       0.78      0.80      0.79      1556
     Neutral       0.76      0.76      0.76      2223
    Positive       0.84      0.82      0.83      1717

    accuracy                           0.79      5496
   macro avg       0.79      0.79      0.79      5496
weighted avg       0.79      0.79      0.79      5496

